In [ ]:
!pip install pandas numpy scikit-learn torch transformers openpyxl

In [ ]:
# Celda 2: Importación de librerías
import pandas as pd
import numpy as np
import unicodedata
import re
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from transformers import AutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

# Celda 3: Funciones de utilidad
def limpiar_texto(texto: str) -> str:
    """Limpia y normaliza texto"""
    if not isinstance(texto, str):
        return ''
    
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

In [ ]:
# Celda 4: Clase para embeddings
class HFEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="dccuchile/bert-base-spanish-wwm-cased", 
                 device=None, max_length=128, batch_size=32):
        self.model_name = model_name
        self.max_length = max_length
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"Usando dispositivo: {self.device}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModel.from_pretrained(self.model_name)
        self.model.to(self.device)
        self.model.eval()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        all_embeddings = []
        total_batches = len(X) // self.batch_size + (1 if len(X) % self.batch_size != 0 else 0)
        
        print(f"Procesando {len(X)} textos en {total_batches} batches...")
        for i in range(0, len(X), self.batch_size):
            print(f"Batch {i//self.batch_size + 1}/{total_batches}", end='\r')
            batch_texts = X[i:i + self.batch_size]
            inputs = self.tokenizer(
                list(batch_texts),
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model(**inputs)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.extend(batch_embeddings)
        
        return np.array(all_embeddings)

In [ ]:
# Celda 5: Función para split de datos
def asegurar_clases_en_entrenamiento(df, y_cols, frac_train=0.70, frac_val=0.15, 
                                   random_state=42):
    """Particiona el dataset garantizando representación de clases"""
    df_train, df_temp = train_test_split(df, test_size=(1-frac_train), 
                                        random_state=random_state)
    
    frac_val_relativo = frac_val / (1-frac_train)
    df_val, df_test = train_test_split(df_temp, test_size=(1-frac_val_relativo), 
                                      random_state=random_state)

    for col in y_cols:
        categorias_entrenamiento = set(df_train[col].unique())
        todas_categorias = set(df[col].unique())
        faltantes = todas_categorias - categorias_entrenamiento
        
        if faltantes:
            print(f"Ajustando categorías faltantes para {col}: {faltantes}")
            for cat in faltantes:
                candidatos = df_val[df_val[col] == cat]
                if not candidatos.empty:
                    df_train = pd.concat([df_train, candidatos.iloc[[0]]])
                    df_val = df_val.drop(candidatos.iloc[[0]].index)
                else:
                    candidatos = df_test[df_test[col] == cat]
                    if not candidatos.empty:
                        df_train = pd.concat([df_train, candidatos.iloc[[0]]])
                        df_test = df_test.drop(candidatos.iloc[[0]].index)

    print(f"\nDistribución final:")
    print(f"Training: {len(df_train)}")
    print(f"Validación: {len(df_val)}")
    print(f"Test: {len(df_test)}")
    
    return df_train, df_val, df_test

# Celda 6: Función para evaluación
def evaluar_modelo(pipeline, X, y, nombre_conjunto='Test'):
    """Evalúa el modelo y muestra métricas por columna"""
    from sklearn.metrics import classification_report
    
    y_pred = pipeline.predict(X)
    
    print(f"\nEvaluación - {nombre_conjunto}:")
    
    for i, col in enumerate(y.columns):
        print(f"\n{col}:")
        print(classification_report(y.iloc[:, i], y_pred[:, i]))


In [ ]:
# Celda 7: Carga y preprocesamiento
# Cargar datos de un solo archivo
df = pd.read_csv("datos.csv")

In [ ]:
# Limpiar texto y manejar NULL
df['SUBLINEA NUEVA'] = df['SUBLINEA NUEVA'].fillna("NULL")

# Limpiar columnas de texto
columnas_a_limpiar = ['Descripcion', 'DepartamentoTroncal', 'DepartamentoSAP', 'Linea']
for col in columnas_a_limpiar:
    df[col] = df[col].fillna('').apply(limpiar_texto)

# Crear texto de entrada
df['TextoEntrada'] = (
    df['Descripcion'] + ' ' +
    df['DepartamentoTroncal'] + ' ' +
    df['DepartamentoSAP'] + ' ' +
    df['Linea']
).str.strip()

# Mantener solo columnas necesarias
cols_necesarias = ['SkuID', 'TextoEntrada', 'DEPARTAMENTO NUEVO', 
                   'SUBDEPARTAMENTO NUEVO', 'LINEA NUEVA', 'SUBLINEA NUEVA']
df = df[cols_necesarias]

# Celda 8: Preparación para entrenamiento
# Definir columnas objetivo
Y_cols = ['DEPARTAMENTO NUEVO', 'SUBDEPARTAMENTO NUEVO', 
          'LINEA NUEVA', 'SUBLINEA NUEVA']

In [ ]:
# Verificar que no queden nulos
print("Filas después de eliminar nulos:", len(df))
print("Nulos restantes por columna:")
print(df.isnull().sum())

In [ ]:
print("Columnas disponibles:", df.columns.tolist())

In [ ]:
print(df.head())

In [ ]:
# Split de datos
df_train, df_val, df_test = asegurar_clases_en_entrenamiento(df, Y_cols)

In [ ]:
# Preparar X e Y
X_train = df_train['TextoEntrada']
Y_train = df_train[Y_cols]

X_val = df_val['TextoEntrada']
Y_val = df_val[Y_cols]

X_test = df_test['TextoEntrada']
Y_test = df_test[Y_cols]

In [ ]:
# Celda 9: Entrenamiento - OPCION INDIVIDUAL
# Crear pipeline
pipeline = Pipeline([
    ('embeddings', HFEmbeddingTransformer(batch_size=16)),
    ('clf', MultiOutputClassifier(LogisticRegression(max_iter=1000)))
])

# Entrenar
print("Iniciando entrenamiento...")
pipeline.fit(X_train, Y_train)

In [ ]:
# Celda 9: Entrenamiento - OPCION GRID
def crear_pipeline_con_grid():
    pipeline = Pipeline([
        ('embeddings', HFEmbeddingTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])
    
    param_grid = {
        'embeddings__max_length': [128, 256],
        'embeddings__batch_size': [8, 16],
        'embeddings__model_name': [
            'dccuchile/bert-base-spanish-wwm-cased',
            'PlanTL-GOB-ES/roberta-base-bne'
        ],
        'clf__estimator__C': [0.1, 1.0, 10.0],
        'clf__estimator__max_iter': [1000, 2000],
        'clf__estimator__class_weight': [None, 'balanced'],
        'clf__estimator__solver': ['lbfgs', 'saga']
    }
    
    return pipeline, param_grid

def ejecutar_grid_search(X_train, y_train, n_jobs=-1, cv=3):
    pipeline, param_grid = crear_pipeline_con_grid()
    
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        n_jobs=n_jobs,
        verbose=2,
        scoring='f1_weighted'
    )
    
    print("Iniciando búsqueda de hiperparámetros...")
    grid_search.fit(X_train, y_train)
    
    print("\nMejores parámetros encontrados:")
    print(grid_search.best_params_)
    print(f"\nMejor puntuación: {grid_search.best_score_:.3f}")
    
    return grid_search.best_estimator_

# Replace the training cell content with:
print("Iniciando búsqueda de hiperparámetros y entrenamiento...")
pipeline = ejecutar_grid_search(X_train, Y_train, n_jobs=4, cv=3)

In [ ]:
# Celda 10: Evaluación
evaluar_modelo(pipeline, X_val, Y_val, 'Validación')
evaluar_modelo(pipeline, X_test, Y_test, 'Test')

In [ ]:
# Celda 11: Guardar modelo (opcional)
import joblib
joblib.dump(pipeline, 'modelo_clasificacion_productos.joblib')

In [ ]:
# Celda 12: Predicción con archivo Excel
def predecir_desde_excel(pipeline, input_file):
   df_pred = pd.read_excel(input_file)
   
   df_pred['TextoEntrada'] = (
       df_pred['Descripcion'].fillna('') + ' ' +
       df_pred['DepartamentoTroncal'].fillna('') + ' ' +
       df_pred['DepartamentoSAP'].fillna('') + ' ' +
       df_pred['Linea'].fillna('')
   ).apply(limpiar_texto)
   
   predicciones = pipeline.predict(df_pred['TextoEntrada'])
   
   for i, col in enumerate(Y_cols):
       df_pred[col] = predicciones[:, i]
   
   # Eliminar columna TextoEntrada
   df_pred.drop('TextoEntrada', axis=1, inplace=True)
   
   output_file = 'datos_predichos.xlsx'
   df_pred.to_excel(output_file, index=False)
   print(f"Predicciones guardadas en: {output_file}")
   
   return df_pred

# Ejecutar predicciones
df_predicciones = predecir_desde_excel(pipeline, 'datos_a_predecir.xlsx')